# Método dos Elementos Finitos - Trabalho 3

Universidade Federal Fluminense

Disciplina ministrada pelo Prof. Marco Ferro

<marcoferro@id.uff.br>

Aluno Noé de Lima

<noe_lima@id.uff.br>

Este trabalho visa aplicar o MEF a uma estrutura de vigas.

Primeiro semestre de 2020

\vfill

A célula a seguir configura o Jupyter-Notebook para exibir as equações matemáticas no formato do ambiente $\LaTeX$ e importa as bibliotecas necessárias.

In [3]:
import numpy as np
import sympy as sym
import json
sym.init_printing(use_latex='mathjax',latex_mode='equation*')
!uname -a

Linux DESKTOP-CR7O8A2 4.19.104-microsoft-standard #1 SMP Wed Feb 19 06:37:35 UTC 2020 x86_64 x86_64 x86_64 GNU/Linux


\cleardoublepage

\tableofcontents

\cleardoublepage

# Introdução

Em uma estrutura de treliças, cada barra é um elemento, e o Método dos Elementos Finitos coincide com a Análise Matricial de Estruturas. Considerando que todos os esforços na treliça são normais, não há cargas transversais, o problema, visto do ponto de vista das barras, é unidimensional, pois, da mesma forma que os esforços, os deslocamentos também são na mesma direção das barras.

No caso de vigas, havendo momento fletor (ou torçor), tanto direto como em consequência de esforço cortante, há deformação no sentido perpendicular à viga, bem como rotação pontual em decorrência da deformação. Assim, uma viga será discretizada na forma de uma malha compostas por pequenos segmentos.

Em um Elemento de Viga, cujo eixo principal é o eixo $x$, temos:

* $V_{1} \rightarrow$ Força perpendicular no início do Elemento;
* $V_{2} \rightarrow$ Força perpendicular no final do Elemento;
* $P \rightarrow$ Carga perpendicular distribuída;
* $M_{1} \rightarrow$ Momento no início do Elemento;
* $M_{2} \rightarrow$ Momento no final do Elemento;
* $L \rightarrow$ Comprimento do Elemento.

Bem como os deslocamentos:
* $u_{1}=\delta_{1}\rightarrow$ deflexão $\delta$ no início do Elemento;
* $u_{2}=\theta_{1}\rightarrow$ inclinação $\theta$ no início do Elemento;
* $u_{3}=\delta_{2}\rightarrow$ deflexão $\delta$ no final do Elemento;
* $u_{4}=\theta_{2}\rightarrow$ inclinação $\theta$ no final do Elemento.

No caso de uma viga retangular de largura $b$ e altura $h$, temos o momento de inércia:

\begin{equation*}
    I = \frac{bh^{3}}{12}
\end{equation*}

Assim, sendo o Módulo de Elasticidade $E$ constante, o produto $EI$ também será.

Temos, portanto, a seguinte equação diferencial da viga:

\begin{equation*}
    EI\frac{d^{4}u}{dx^{4}} = P
\end{equation*}

Tem-se que:

\begin{equation*}
    V = EI\frac{d^{3}u}{dx^{3}}
\end{equation*}

E

\begin{equation*}
    M = EI\frac{d^{2}u}{dx^{2}}
\end{equation*}

Aplicando a SRP - Sentença de Resíduos Ponderados - na EDO da viga, resulta:

\begin{equation*}
    \int_{\Omega}{N_{i}R}d\Omega = 0
\end{equation*}

$$\therefore$$

\begin{equation*}
    \int_{0}^{L}{N_{i}\left(EI\frac{d^{4}u}{dx^{4}}-P\right)}dx = 0
\end{equation*}


In [45]:
x = sym.symbols('x')
P = sym.symbols('P')
E,I,L = sym.symbols('E,I,L')
u = sym.Function('u')
N_i = sym.symbols('N_i',cls=sym.Function)
R = E*I*sym.diff(u(x),x,4)-P
SRP = sym.integrate(N_i(x)*R,(x,0,L))
display(SRP.doit())

L                             
⌠                             
⎮ ⎛      4          ⎞         
⎮ ⎜     d           ⎟         
⎮ ⎜E⋅I⋅───(u(x)) - P⎟⋅Nᵢ(x) dx
⎮ ⎜      4          ⎟         
⎮ ⎝    dx           ⎠         
⌡                             
0                             

Essa integral não pode ser resolvida pelos métodos sym.dsolve(sym.Eq(SRP,0),u(x)) ou SRP.doit(), pois o nível de complexidade ainda não foi implementado na biblioteca.

Utilizando a integração por partes, obtém-se o seguinte:

In [47]:
eq = sym.integrate(E*I*(sym.diff(N_i(x),x)*sym.diff(u(x),x,3)),(x,0,L))
display(eq.doit())

    L                       
    ⌠                       
    ⎮             3         
    ⎮ d          d          
E⋅I⋅⎮ ──(Nᵢ(x))⋅───(u(x)) dx
    ⎮ dx          3         
    ⎮           dx          
    ⌡                       
    0                       

In [62]:
N_1,N_2,N_3,N_4 = sym.symbols('N_1,N_2,N_3,N_4',cls=sym.Function)
#display(sym.dsolve(sym.Eq(sym.diff(N_1(x),x,2),0),ics={N_1(0):1,N_1(L):0}))
#display(sym.dsolve(sym.Eq(sym.diff(N_2(x),x,2),0),ics={N_2(x).diff(x).subs(x, L):1,N_2(x).diff(x).subs(x, L):0}))
#display(sym.dsolve(sym.Eq(sym.diff(N_3(x),x,2),0),ics={N_3(0):0,N_3(L):1}))
#display(sym.dsolve(sym.Eq(sym.diff(N_4(x),x,2),0),ics={N_4(x).diff(x).subs(x, L):1,N_4(x).diff(x).subs(x, L):0}))

            x
N₁(x) = 1 - ─
            L

N₂(x) = C₁

        x
N₃(x) = ─
        L

N₄(x) = C₁

In [96]:
K_e = sym.Matrix([[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]])

K_e[0,0] = 12*E*I/L**3
K_e[0,1] = 6*E*I/L**2
K_e[0,2] = -12*E*I/L**3
K_e[0,3] = 6*E*I/L**2

K_e[1,0] = K_e[0,1]
K_e[1,1] = 4*E*I/L
K_e[1,2] = -6*E*I/L**2
K_e[1,3] = 2*E*I/L

K_e[2,0] = K_e[0,2]
K_e[2,1] = K_e[1,2]
K_e[2,2] = 12*E*I/L**3
K_e[2,3] = -6*E*I/L**2

K_e[3,0] = K_e[0,3]
K_e[3,1] = K_e[1,3]
K_e[3,2] = K_e[2,3]
K_e[3,3] = 4*E*I/L

display(K_e/(E*I/L))

⎡ 12    6   -12    6 ⎤
⎢ ──    ─   ────   ─ ⎥
⎢  2    L     2    L ⎥
⎢ L          L       ⎥
⎢                    ⎥
⎢ 6         -6       ⎥
⎢ ─     4   ───    2 ⎥
⎢ L          L       ⎥
⎢                    ⎥
⎢-12   -6    12   -6 ⎥
⎢────  ───   ──   ───⎥
⎢  2    L     2    L ⎥
⎢ L          L       ⎥
⎢                    ⎥
⎢ 6         -6       ⎥
⎢ ─     2   ───    4 ⎥
⎣ L          L       ⎦

In [89]:
V_1,M_1,V_2,M_2 = sym.symbols('V_1,M_1,V_2,M_2')
f_e = sym.Matrix([V_1,-M_1,-V_2,M_2])

f_e[0] += P*L/2
f_e[1] += P*L**2/12
f_e[2] += P*L/2
f_e[3] += -P*L**2/12

display(f_e)

⎡ L⋅P       ⎤
⎢ ─── + V₁  ⎥
⎢  2        ⎥
⎢           ⎥
⎢  2        ⎥
⎢ L ⋅P      ⎥
⎢ ──── - M₁ ⎥
⎢  12       ⎥
⎢           ⎥
⎢ L⋅P       ⎥
⎢ ─── - V₂  ⎥
⎢  2        ⎥
⎢           ⎥
⎢   2       ⎥
⎢  L ⋅P     ⎥
⎢- ──── + M₂⎥
⎣   12      ⎦